<a href="https://colab.research.google.com/github/FudgeSato/Quant-Backtesting-Monte-Carlo/blob/main/General_Portfolio_Backtester_and_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf

# Step 1: Custom Portfolio Selection
print("Enter stock tickers (comma-separated, e.g., AAPL,MSFT,GOOGL):")
tickers_input = input()
tickers = [ticker.strip() for ticker in tickers_input.split(",")]  # Strip whitespace from each ticker

# User selects weighting method
print("Enter 'equal' for equal-weighted portfolio or 'custom' to input your own allocations:")
weight_method = input().strip().lower()

if weight_method == "custom":
    print(f"Enter weights for {len(tickers)} stocks (comma-separated, must sum to 1):")
    weights_input = input()  # Get user input for weights
    weights = np.array([float(w.strip()) for w in weights_input.split(",")])  # Strip spaces and convert to floats
    if not np.isclose(weights.sum(), 1.0):
        raise ValueError("Weights must sum to 1.")
else:
    weights = np.ones(len(tickers)) / len(tickers)  # Default: Equal-weighted

# User selects benchmarks
print("Enter benchmark tickers (comma-separated, e.g., SPY,QQQ,BND,AOR,VTI,VT) or press Enter for none:")
benchmark_input = input().strip()
benchmarks = [ticker.strip() for ticker in benchmark_input.split(",")] if benchmark_input else []

all_tickers = tickers + benchmarks

# Adjust the end date to a date in the past
data = yf.download(all_tickers, start="2021-01-01", end="2025-03-11")["Close"]
data = data.ffill().bfill()  # Handle missing values

# Check if 'Ticker' column exists and use it if necessary
if 'Ticker' in data.columns:
    data = data.set_index('Ticker').T  # Transpose to have tickers as columns

# Step 2: Compute Returns and Portfolio Performance
returns = data.pct_change().dropna()
portfolio_returns = returns[tickers] @ weights
cumulative_returns = (1 + portfolio_returns).cumprod() * 10000  # Starting at $100

# Compute benchmark cumulative returns
if benchmarks:
    benchmark_cumulative_returns = (1 + returns[benchmarks]).cumprod() * 10000

# Step 4: Risk/Reward Analysis
annualized_return = (portfolio_returns.mean() * 252)
volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratio = (annualized_return - 0.02) / volatility  # 2% risk-free rate

# Step 5: Interactive Visualization
fund_name = input("Enter the name of your portfolio: ")
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name=f'{fund_name}', line=dict(width=3)))  # Line width changed here
if benchmarks:
    for benchmark in benchmarks:
        fig.add_trace(go.Scatter(x=benchmark_cumulative_returns.index, y=benchmark_cumulative_returns[benchmark], mode='lines', name=benchmark))
# Add annotations for key metrics
fig.add_annotation(
    x=0.05,  # Adjust x position as needed
    y=0.95,  # Adjust y position as needed
    xref="paper",  # Relative to the plot area
    yref="paper",  # Relative to the plot area
    text=f"Annualized Return: {annualized_return:.2%}<br>Volatility: {volatility:.2%}<br>Sharpe Ratio: {sharpe_ratio:.2f}",
    showarrow=False,  # No arrow
    font=dict(size=12, color="white"),  # Adjust font as needed
    bgcolor="rgba(0, 0, 0, 0.5)",  # Semi-transparent background
    bordercolor="white",  # Border color
    borderwidth=1  # Border width
)

fig.update_layout(
    title={
        'text': f"{fund_name} Portfolio Performance vs Benchmarks",
        'subtitle': {'text': f"Your portfolio's performance over time ({tickers_input})"}
    },
    xaxis_title="Date", yaxis_title="Value ($)", template="plotly_dark"
)
fig.show()


# Print key metrics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Volatility: {volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Enter stock tickers (comma-separated, e.g., AAPL,MSFT,GOOGL):
NVDA,NOVO-B.CO,BAM,BRK-B,TSM,TSLA,LLY,ASML,COST,WMT,NFLX,BX,AVGO,KKR,DLR,APO,EQIX,VST
Enter 'equal' for equal-weighted portfolio or 'custom' to input your own allocations:
custom
Enter weights for 18 stocks (comma-separated, must sum to 1):
0.300908,0.147954,0.123197,0.057660,0.057118,0.055227,0.046809,0.043703,0.039085,0.025210,0.023747,0.017570,0.014900,0.012223,0.012212,0.010593,0.007271,0.004613
Enter benchmark tickers (comma-separated, e.g., SPY,QQQ,BND,AOR,VTI,VT) or press Enter for none:
SPY, GLD
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  20 of 20 completed


Enter the name of your portfolio: Titan Ashura Smart Beta


Annualized Return: 34.76%
Volatility: 25.75%
Sharpe Ratio: 1.27


In [2]:
# @title
# prompt: calculate and plot the 50 and 200 day SMA of the portfolio, scale the SMA lines to the NAV of the portfolio, and only plot the scaled lines against the NAV

# Calculate 50-day and 200-day SMAs
cumulative_returns.index = pd.to_datetime(cumulative_returns.index)
sma50 = cumulative_returns.rolling(window=50).mean()
sma200 = cumulative_returns.rolling(window=200).mean()

# Scale SMA lines to NAV
sma50_scaled = sma50 / cumulative_returns.iloc[-1] * cumulative_returns.iloc[-1]
sma200_scaled = sma200 / cumulative_returns.iloc[-1] * cumulative_returns.iloc[-1]


# Plotting
fig = go.Figure()

# Add NAV
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name=f'{fund_name}', line=dict(width=3)))

# Add scaled SMAs
fig.add_trace(go.Scatter(x=sma50_scaled.index, y=sma50_scaled, mode='lines', name='50-Day SMA (Scaled)', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=sma200_scaled.index, y=sma200_scaled, mode='lines', name='200-Day SMA (Scaled)', line=dict(color='purple')))


# Update layout (similar to your existing layout)
fig.update_layout(
    title={
        'text': f"{fund_name} Portfolio Performance with SMAs",
        'subtitle': {'text': f"Your portfolio's performance over time ({tickers_input})"}
    },
    xaxis_title="Date", yaxis_title="Value ($)", template="plotly_dark"
)
fig.show()


In [3]:
# @title
import plotly.graph_objects as go
# Step 3: Monte Carlo Simulation with TensorFlow for GPU Acceleration
num_simulations = 10000  # Scalable to 1 million
num_days = len(portfolio_returns)
batch_size = 10000  # Run simulations in batches to avoid VRAM overload

# Limit VRAM usage to 10GB
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    except RuntimeError as e:
        print(e)

# Convert data to TensorFlow tensors
portfolio_returns_tf = tf.convert_to_tensor(portfolio_returns.values, dtype=tf.float32)

def monte_carlo_simulation():
    random_returns = tf.random.shuffle(portfolio_returns_tf, seed=None)
    return tf.math.cumprod(1 + random_returns, axis=0) * 10000

# Run Monte Carlo in batches if VRAM is exhausted
simulated_nav = []
for _ in range(num_simulations // batch_size):
    batch_result = tf.map_fn(lambda _: monte_carlo_simulation(), tf.range(batch_size), dtype=tf.float32)
    simulated_nav.append(batch_result.numpy())

simulated_nav = np.concatenate(simulated_nav, axis=1)

# Step 6: 1-Year Future Monte Carlo Simulation
future_days = 252

# Generate future random returns based on historical distribution
future_simulations = np.random.choice(portfolio_returns, size=(future_days, num_simulations))
future_cumulative_returns = np.cumprod(1 + future_simulations, axis=0) * 10000

# Calculate max drawdown
def max_drawdown(returns):
    cumulative = np.cumprod(1 + returns, axis=0)
    peak = np.maximum.accumulate(cumulative, axis=0)
    drawdown = (cumulative - peak) / peak
    return drawdown.min(axis=0)

future_max_drawdowns = max_drawdown(future_simulations)

# Calculate percentiles of returns
percentiles = np.percentile(future_cumulative_returns[-1, :], [25, 50, 75, 90])
mean_return = np.mean(future_cumulative_returns[-1, :])

# Interactive plot using Plotly
fig = go.Figure()
for i in range(100):  # Plot only 100 paths for visibility
    fig.add_trace(go.Scatter(x=list(range(future_days)), y=future_cumulative_returns[:, i],
                             mode='lines', line=dict(width=1, color='red'), opacity=0.1, showlegend=False))

fig.update_layout(
    title=f"{fund_name} Portfolio: 1-Year Future Monte Carlo Simulation",
    xaxis_title="Days into the Future",
    yaxis_title="Future NAV",
    template="plotly_dark"
)
fig.show()

# Print statistics
print(f"Max Drawdown: {np.min(future_max_drawdowns):.2%}")
print(f"25th Percentile Return: {percentiles[0]:.2f}")
print(f"50th Percentile (Median) Return: {percentiles[1]:.2f}")
print(f"Mean Return: {mean_return:.2f}")
print(f"75th Percentile Return: {percentiles[2]:.2f}")
print(f"90th Percentile Return: {percentiles[3]:.2f}")


Max Drawdown: -57.09%
25th Percentile Return: 11514.46
50th Percentile (Median) Return: 13706.99
Mean Return: 14162.39
75th Percentile Return: 16373.27
90th Percentile Return: 19107.31
